In [ ]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import tensorflow as tf
import random as rn
from keras import backend as K
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

from Match_SOM import MATCHSOM
from utility import neuron_idx, idx_convert
from metrics import aveNN_ATT
from sklearn.model_selection import train_test_split
import pickle
import keras
import time

In [ ]:
ihdp_tr = np.load('../data/ihdp_npci_1-100.train.npz')
ihdp_te = np.load('../data/ihdp_npci_1-100.test.npz')

In [ ]:
X_tr, T_tr, O_tr = ihdp_tr['x'],  ihdp_tr['t'], ihdp_tr['yf']
X_te, T_te, O_te = ihdp_te['x'],  ihdp_te['t'], ihdp_te['yf']

In [ ]:
col_name = []
for i in range(25):
    col_name.append('feat{}'.format(i+1))
col_name = col_name + ['assignment', 'outcome']  

In [ ]:
start = time.time()

ATT = []
#testloss = []

for i in range(100):   
    
    # training data
    Xtrain,ytrain,o_tr = X_tr[:,:,i],T_tr[:,i],O_tr[:,i]
    Xtest,ytest,o_te = X_te[:,:,i],T_te[:,i],O_te[:,i]
    data = np.vstack((Xtrain,Xtest))
    t = np.expand_dims(np.append(ytrain,ytest),axis=1).astype(int)
    outcome = np.expand_dims(np.append(o_tr,o_te),axis=1)
    xty = pd.DataFrame(np.append(np.append(data,t,axis=1),outcome,axis=1),columns = col_name)
    
    # set a seed
    np.random.seed(0)
    rn.seed(0)
    config = tf.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
    tf.set_random_seed(0)
    sess = tf.Session(graph=tf.get_default_graph(), config=config)
    K.set_session(sess)
    
    side = int(round(np.sqrt(5*np.sqrt(xty.shape[0])))) 
    map_size=(side,side)  
    neuron_map = neuron_idx(map_size)  
    
    # Initialize model
    matchsom = MATCHSOM(encoder_dims=[Xtrain.shape[-1], 2], map_size=map_size)    
    optimizer = keras.optimizers.Adam(learning_rate=0.0001) 
    matchsom.initialize(ae_act='relu', ae_init='glorot_uniform')
    matchsom.compile(gamma=1, optimizer=optimizer)
    #testloss.append( matchsom.pretrain(X = Xtrain, y = ytrain, Xval = Xtest,yval = ytest, epochs= 1000))
    # fitting 
    matchsom.fit(X_train = Xtrain, y_train=ytrain,
                X_val=Xtest, y_val=ytest,
                iterations=3000,
                som_iterations=3000,
                eval_interval=100,
                save_epochs=10,
                batch_size=64,
                Tmax= 10,
                Tmin=0.1,
                decay= 'exponential')
        
    #  evaluate on the lalonde rct part data
    _, distance = matchsom.model.predict(data)
    bmu = np.argmin(distance,axis=1)
    bmu_idx = idx_convert(bmu,neuron_map)
    ta = aveNN_ATT(xty,bmu_idx,'cityblock')
    ATT.append(ta)
    

dur = time.time()- start
print(dur)